# Projeto Final - Sistema de Controle Financeiro

Deverá ser desenvolvido um sistema para controle financeiro que receba as movimentações e as armazena em um arquivo csv ou json.

O sistema deverá ser capaz de realizar as seguintes operações:

- **Criar** novos registros e identificar a data que o registro foi feito, qual tipo de movimentação, valor.

- Os tipos podem ser despesas, receita ou investimento:
  - No caso de receita, o valor deve ser tratado como numerico e armazenado normalmente.
  - no caso de despesas o valor deve ser recebido como positivo, mas armazenado como negativo
  - No caso de investimento, deve ter uma informação a mais de 'Montante', em que será calculado quanto o dinheiro rendeu desde o dia que foi investido. Para essa finalidade utilize a seguinte formula: $M = C * (1 + i)^t$ ([Saiba mais](https://matematicafinanceira.org/juros-compostos/)), considere tudo em dias.
- **Ler** registros: Deverá ser possível consultar os registros por data, tipo ou valor.
- **Atualizar** registros: No caso de atualização, pode-se atualizar o valor, o tipo e a data deverá ser a de atualização do registro.
- **Deletar**: Deverá ser possível deletar o registro (caso necessário, considere o indice do elemento como ID)
- Crie uma função que atualize os valores de rendimento sempre que chamada
- Crie uma função exportar_relatorio, que seja possível exportar um relatorio final em csv ou json .
- Crie pelo menos uma função de agrupamento, que seja capaz de mostrar o total de valor baseado em alguma informação (mes, tipo...)
- Crie valores separados para identificar a data (dia, mes, ano)

---

Atenção:
- Não utilize a biblioteca pandas para resolução desse exercício
- Deem um nome criativo para a aplicação de vocês

# Nome Criativo para a aplicação

Descrever como utilizar, e o que cada função faz

## Entendendo o problema

### Como será a entrada dos dados?
  * O usuário pode entrar com uma lista de depositos?
  * Pode ser entradas uma a uma, informando a data, valor, e tipo de operação?
  * O programa vai ler um arquivo json ou csv com todas as movimentações feitas em um determindo periodo?

## Possibiliades

Considerando que, independentemente da forma de entrada dos dados, temos que gerar uma lista de dicionários, em que cada dicionário vai conter obrigatoriamente a informação de "valor", "tipo" e "data", podemos montar a lógica do programa da seguinte forma:

### Estrutura dos dados
- **Registro**: Cada movimentação será um dicionário com as seguintes chaves:
    - **`data`**: Uma tupla (dia, mês, ano), ou string representando a data do registro.
    - **`tipo`**: Uma string que pode ser "despesa", "receita" ou "investimento".
    - **`valor`**: Um número representando o valor da movimentação.
    - **`montante`**: Um número representando o montante acumulado.

### Funções principais

- **Criar Registro/Movimentação**: Adiciona um novo registro ao arquivo de dados.
  * Pode ler de uma lista de dicionarios ou por inputs individuais
- **Ler Registros**: Permite a consulta por data, tipo ou valor.
- **Atualizar Registro**: Modifica um registro existente.
- **Deletar Registro**: Remove um registro.
- **Calcula Rendimento**Ç Calcula o montante de um investimento usando a fórmula M=C*(1+i)^t.
- **Calcular Montante de Investimento**: Usa a função anterior e calcula o rendimento para cada intervalo de dadas dos registros.
- **Exportar Relatório (Extrato bancario?)**: Gera um relatório em formato CSV ou JSON. Ordenado por data de deposito?
- **Função de Agrupamento**: Agrega os valores com base em uma categoria específica (mês, tipo, etc.).

Importar modulos

In [ ]:
import csv
import json
import datetime
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def read_csv(path):
    """
    retorna uma lista de listas, onde a primeira lista é o header,
    e as outras correspondem aos valores de cada linha do arquivo.
    """
    with open(path, newline='') as f:
        reader = csv.reader(f)
        linhas = []
        for row in reader:
            linhas.append(row)

    return linhas

def cria_conta_bancaria(data_criacao="01/01/1970"):
   return [{"data": datetime.datetime.strptime(data_criacao, "%d/%m/%Y"),
           "conta_corrente": 0,
           "investimentos": 0,
           "rendimento": 0,
           "montante_investimento": 0,
           "tipo":"receita",
           "valor": 0}]

def criar_registro_movimentacao(id, data, tipo, valor):
    if tipo == 'despesa':
        valor = -abs(valor)  # Garante que despesa seja armazenada como negativa
    return {'ID': id,
            'data': data,
            'tipo': tipo,
            'valor': valor}


def movimentacao(input_terminal=True, lista_movimentacoes=None):
    movimentacao = []
    if input_terminal:
      entrada = 1
      id = 0
      while entrada !=0:
          data = input("Digite a data da movimentacao (dd/mm/aaa): ")
          tipo = input("Qual o tipo da movimentacao, receita, despesa ou investimento? ")
          valor = abs(float(input("Digite o valor da movimentacao: "))) #garante que o valor seja positivo para qualquer opera nesse momento.
          entrada = int(input("Mais alguma movimentação? Digite 1 para continuar ou 0 para sair. "))
          id += 1
          movimentacao.append(criar_registro_movimentacao(id, datetime.datetime.strptime(data, "%d/%m/%Y"), tipo, valor))
    else:
        moviment = read_csv(lista_movimentacoes)[1:]
        for id, mm in enumerate(moviment):
            # movimentacao.append(criar_registro_movimentacao(datetime.datetime.strptime(mm[0], "%Y-%m-%d"), mm[1], float(mm[2])))
            movimentacao.append(criar_registro_movimentacao(id, datetime.datetime.strptime(mm[0], "%d/%m/%Y"), mm[1], float(mm[2])))

    return movimentacao

def deletar_registro(indice, movimentacoes):
    if indice < len(movimentacoes):
        del movimentacoes[indice]

    return movimentacoes


def calcular_rendimento(taxa=0.003, valor=0, data_anterior=None, data_atual=None):
    # Cálculo do rendimento de investimento
    # M = C * (1 + i)^t
    # t = contar_dias_entre_datas(data_anterior,data_atual)
    t = (data_atual - data_anterior).days
    montante_final = valor * (1 + taxa)**t
    rendimento = montante_final- valor
    return rendimento

def calcular_montante(lista_depositos):
    for i in range(len(lista_depositos)):
        if i == 0:
            rendimento = 0
            montante = lista_depositos[0]["investimentos"]
        else:
            montante_anterior = lista_depositos[i-1]["montante_investimento"]
            investido_agora = lista_depositos[i]["investimentos"]
            rendimento = calcular_rendimento(taxa=0.001,
                                             valor=montante_anterior,
                                             data_anterior=lista_depositos[i-1]["data"],
                                             data_atual=lista_depositos[i]["data"])
            montante = rendimento + investido_agora + montante_anterior

        lista_depositos[i]["rendimento"] = rendimento
        lista_depositos[i]["montante_investimento"] = montante

    return lista_depositos


def extrato_conta(movimentacoes):
    sorted_list = sorted(movimentacoes, key=lambda x: x['data'])
    extrato = cria_conta_bancaria()
    investiment0 = 0
    cc = 0
    for i, mvm in enumerate(sorted_list):
        if mvm['tipo'] in ["receita", "despesa"]:
            cc = mvm["valor"]
            investimento = 0
        else:
            investimento = mvm["valor"]
            cc = 0

        extrato.append({"ID": mvm["ID"],"data": mvm["data"],
                            "conta_corrente": cc,
                            "investimentos": investimento,
                            "rendimento": 0,
                            "montante_investimento": 0,
                            "tipo": mvm['tipo'],
                            "valor": mvm['valor']})

    extrato = calcular_montante(extrato)
    return extrato


def atualizar_registro(movimentacoes, indice, valor=None, tipo=None):
    if indice < len(movimentacoes):
        if valor is not None:
            movimentacoes[indice]['valor'] = valor
        if tipo is not None:
            movimentacoes[indice]['tipo'] = tipo
    return movimentacoes

def exportar_relatorio(movimentacoes, formato='json'):
    if formato == 'json':
        return json.dumps(movimentacoes, default=str, indent=4)
    elif formato == 'csv':
        relatorio_csv = "Data,Tipo,Valor,Montante\n"
        for mov in movimentacoes[1:]:
            relatorio_csv += f"{mov['data']},{mov['tipo']},{mov['valor']},{mov.get('montante_investimento', '')}\n"
        return relatorio_csv

def agrupar_por_mes(movimentacoes):
    agrupado = {}
    for mov in movimentacoes:
        mes = mov['data'].strftime("%Y-%m")
        if mes in agrupado:
            agrupado[mes] += mov['valor']
        else:
            agrupado[mes] = mov['valor']
    return agrupado

In [ ]:

lista = "/content/drive/MyDrive/Colab Notebooks/ada_SantanderCoders_EngDados/Logica Programacao/projeto/financas_aleatorias_atualizado.csv"
movimentacoes = movimentacao(input_terminal=False, lista_movimentacoes=lista)
#

In [ ]:
extrato = extrato_conta(movimentacoes)

In [ ]:
relatorio = exportar_relatorio(extrato, 'json')

In [ ]:
extrato

In [ ]:
teste_data

In [ ]:
def movimentacao(input_terminal=True, lista_movimentacoes=None):
    movimentacao = []
    if input_terminal:
      entrada = 1
      while entrada !=0:
          data = input("Digite a data da movimentacao (dd/mm/aaa): ")
          tipo = input("Qual o tipo da movimentacao, receita, despesa ou investimento? ")
          valor = abs(float(input("Digite o valor da movimentacao: "))) #garante que o valor seja positivo para qualquer opera nesse momento.
          entrada = int(input("Mais alguma movimentação? Digite 1 para continuar ou 0 para sair. "))

          movimentacao.append(criar_registro_movimentacao(datetime.datetime.strptime(data, "%d/%m/%Y"), tipo, valor))
    else:
        moviment = read_csv(lista_movimentacoes)[1:]
        for mm in moviment:
            movimentacao.append(criar_registro_movimentacao(mm[0], mm[1], float(mm[2])))

    return movimentacao

In [ ]:
movimentacoes = movimentacao(input_terminal=False, lista_movimentacoes=lista)

In [ ]:
movimentacoes

In [ ]:
sortesd = sorted(movimentacoes, key=lambda x: x['data_numero'])

In [ ]:
contar_dias_entre_datas(sortesd[2]['data'], sortesd[3]['data'])

In [ ]:
sortesd